In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import image
import binascii
from PIL import Image


In [3]:
Photo1 = Image.open("Photo1.jpg")
Photo2 = Image.open('Photo2.jpg')

hiddenMessage = open('hiddenMessage.txt', 'r').read()
hiddenMessage = str(hiddenMessage)


In [4]:
def get_binary(message):
    encodeM = message.encode()
    bin_int = int.from_bytes(encodeM, 'big')
    bin_string = bin(bin_int)[2:]
    return "0" + bin_string

In [16]:
def implant_message(pixels, binString):
    # loop through each pixel
    for i in range(len(pixels)):
        for j in range(len(pixels[i])):
            
            if (binString == ''):
                break
            # print(binString[:2])
            # get the pixel value this is in binary already
            pixel = pixels[i][j]
            # get first two characters of binString to be implanted into pixel
            binStringC = binString[:2]
            # remove first two characters of binString to the message will be eaten while looping
            binString = binString[2:]
            # get the first 6 character of binC then add the two bits of binStringC to the end
            pixel = pixel[:6] + binStringC
            #set the new pixel value
            pixels[i][j] = pixel
    return pixels

                
                

In [6]:
def retrieve_message(pixels):
    # create empty list of binary values
    binList = []
    # loop through each pixel
    for i in range(len(pixels)):
        for j in range(len(pixels[i])):
            # get the pixel value
            pixel = pixels[i][j]
            # get the last 2 characters of pixel
            binC = pixel[-2:]
            # add the the bits to the list
            binList.append(binC)
    #         # get the pixel value
    #         pixel = img[i, j]
    #         # split the pixel value into a list of RGB values
    #         rgb = pixel.tolist()
    #         # loop through each RGB value
    #         for k in range(len(rgb)):
    #             # convert RGB value to decimal and return binary value
    #             binC = bin(rgb[k])[2:]
    #             # get last two characters of binC
    #             binC = binC[-1:]
    #             # add the binary value to the list
    #             binList.append(binC)
    l = ''.join(binList)
    # l = '0'? + l
    return l

In [7]:
def binary_to_string(binString):
    binary_int = int(binString, 2)  
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    return ascii_text

In [8]:
#this method is used to get the lentgh of message that the user can hide
def get_length(img):
    # create empty list of binary values
    binList = []
    # loop through each pixel
    for i in range(len(img)):
        for j in range(len(img[i])):
            # get the pixel value
            pixel = img[i, j]
            # split the pixel value into a list of RGB values
            rgb = pixel.tolist()
            # loop through each RGB value
            for k in range(len(rgb)):
                # convert RGB value to decimal and return binary value
                binC = bin(rgb[k])[2:]
                # get last two characters of binC
                binC = binC[-2:]
                # add the binary value to the list
                binList.append(binC)
    return len(binList)

In [9]:
pixels = Photo1.getdata()
test = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixels]
# test

In [14]:
# newPhotoPixelsBinary


# Hide info and create photo

In [30]:
binMessage = get_binary(hiddenMessage)
# binMessage = '0' + binMessage
# turn photo to pixel representation
pixels = Photo1.getdata()
# turn pixel representation into a list of lists
binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixels]
newPhotoPixelsBinary = implant_message(binaryPixels, binMessage)
newPhotoPixels = [tuple(int(p,2) for p in pixel) for pixel in newPhotoPixelsBinary]
newPhoto = Image.new("RGB", Photo1.size)
newPhoto.putdata(newPhotoPixels)
finalFilename = ".".join(["test", "png"])
newPhoto.save(finalFilename)
# newPhoto.show()
# test = newPhoto.getdata()



In [22]:
newPhotoPixels

[(109, 65, 61),
 (104, 57, 54),
 (102, 56, 49),
 (102, 57, 53),
 (104, 62, 52),
 (104, 61, 55),
 (105, 57, 53),
 (106, 63, 54),
 (105, 62, 52),
 (105, 61, 54),
 (107, 62, 53),
 (106, 62, 53),
 (105, 62, 55),
 (105, 61, 54),
 (107, 63, 53),
 (107, 61, 53),
 (113, 63, 48),
 (115, 60, 50),
 (112, 60, 49),
 (112, 61, 48),
 (109, 62, 53),
 (109, 61, 54),
 (108, 63, 53),
 (110, 63, 52),
 (113, 62, 52),
 (109, 61, 55),
 (108, 62, 53),
 (110, 60, 53),
 (109, 63, 53),
 (108, 61, 54),
 (110, 61, 53),
 (110, 63, 55),
 (113, 62, 51),
 (114, 60, 50),
 (112, 60, 49),
 (114, 63, 51),
 (113, 62, 49),
 (118, 60, 54),
 (116, 60, 53),
 (119, 61, 52),
 (117, 58, 50),
 (116, 57, 50),
 (117, 57, 48),
 (118, 56, 48),
 (117, 59, 49),
 (116, 57, 50),
 (118, 56, 49),
 (118, 58, 49),
 (109, 63, 44),
 (114, 61, 47),
 (113, 60, 45),
 (114, 61, 49),
 (113, 62, 49),
 (113, 61, 50),
 (115, 62, 52),
 (114, 60, 52),
 (113, 63, 57),
 (113, 61, 58),
 (111, 62, 53),
 (110, 62, 57),
 (105, 63, 57),
 (100, 61, 58),
 (101, 6

In [21]:
newPhotoPixels
binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in newPhotoPixels]
# binaryPixels


In [31]:
hiddenMessagePhoto = Image.open('test.png')
# binaryPixels

In [39]:
# turn photo to pixel representation
pixelsWithMessage = hiddenMessagePhoto.getdata()
# turn pixel representation into a list of lists
binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixelsWithMessage]
binaryPixels
pixelsWithMessage
test = retrieve_message(binaryPixels)
test
p = return_message(test)
p


'The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature\'s God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.\nWe hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundati

In [35]:
# newPhoto

In [36]:
def return_message(msg):
    test = [msg[i:i+8] for i in range(0, len(msg), 8)]
    # newArray = np.zeros((len(test), len(test[0]), 3), dtype=np.uint8)
    apple = ""
    for t in test:
        apple += chr(int(t, 2))
        # chr(int(t, 2)), end=""
    return apple

In [30]:
a = return_message(binMessage)
# print(a)

In [97]:
b = return_message(newMessage)
# print(b)

In [12]:
binMessage = get_binary(hiddenMessage)
binMessage
binMessage = '0' + binMessage
test = [binMessage[i:i+8] for i in range(0, len(binMessage), 8)]
test[:1]

['01010100']

In [94]:
newArray = np.zeros((len(test), len(test[0]), 3), dtype=np.uint8)
apple = ""
for t in test:
    apple += chr(int(t, 2))
    # chr(int(t, 2)), end=""
# apple

In [98]:
newMessage = retrieve_message(newPhoto)
# newMessage = "0" + newMessage
test = [newMessage[i:i+8] for i in range(0, len(newMessage), 8)]
test[0:12]

['01010100',
 '01101000',
 '01100101',
 '00100000',
 '01110101',
 '01101110',
 '01100001',
 '01101110',
 '01101001',
 '01101101',
 '01101111',
 '01110101']

In [136]:
c = 64736/8
print(c)
a = test[0:int(c)]
# for t in a:


8092.0


In [34]:
#covert binary character to ascii character
def binary_to_ascii(binary):
    return chr(int(binary,2))
|
print(binary_to_ascii(a))

SyntaxError: invalid syntax (3204754380.py, line 4)

In [35]:
for t in test:
    print(binary_to_ascii(t))

NameError: name 'binary_to_ascii' is not defined

In [93]:
# print(binMessage)

In [38]:
# newHiddenMessage = binary_to_string(newMessage)
binary_int = int(binM, 2)  
byte_number = binary_int.bit_length() + 7 // 8
binary_array = binary_int.to_bytes(byte_number, "big")
ascii_text = binary_array.decode()
print(ascii_text)

NameError: name 'binM' is not defined

In [39]:
byte_array = "$$$".encode()


binary_int = int.from_bytes(byte_array, "big")

binary_string = bin(binary_int)[2:]


print(binary_string)


1001000010010000100100


In [135]:


binString = '01010110'
rgb = ['01101111', '01000010', '00111100']
print("rgb" + str(rgb))
# loop through each RGB value
for k in range(len(rgb)):
    # convert RGB value to decimal and return binary value
    binC = rgb[k]
    print("binC " + str(binC))
    # get first two characters of binString
    binStringC = binString[:2]
    print("binStringC: " + binStringC)
    # remove first two characters of binString
    binString = binString[2:]
    print("binString after remove first twp: " + binString)
    # get the first 7 character of binC
    binC = binC[:6] + binStringC
    print("new binC: " + binC)
    # set RGB value to the new binary value
    rgb[k] = binC
    print("new rgb value: " + str(int(rgb[k])))
print(rgb)

rgb['01101111', '01000010', '00111100']
binC 01101111
binStringC: 01
binString after remove first twp: 010110
new binC: 01101101
new rgb value: 1101101
binC 01000010
binStringC: 01
binString after remove first twp: 0110
new binC: 01000001
new rgb value: 1000001
binC 00111100
binStringC: 01
binString after remove first twp: 10
new binC: 00111101
new rgb value: 111101
['01101101', '01000001', '00111101']


In [44]:
bin_String = ""
rgb = ['01101111', '01000010', '00111100']
print("rgb" + str(rgb))
# loop through each RGB value
for k in range(len(rgb)):
    # convert RGB value to decimal and return binary value
    binC = rgb[k]
    print("binC " + str(binC))
    # get the last two characters of binC
    binC = binC[-2:]
    print("binC last two: " + binC)
    # add the binary value to the list
    bin_String += binC
print(bin_String)

    # # get first two characters of binString
    # binStringC = binString[:2]
    # print("binStringC: " + binStringC)
    # # remove first two characters of binString
    # binString = binString[2:]
    # print("binString after remove first twp: " + binString)
    # # get the first 7 character of binC
    # binC = binC[:6] + binStringC
    # print("new binC: " + binC)
    # # set RGB value to the new binary value
    # rgb[k] = binC
    # print("new rgb value: " + str(int(rgb[k])))
# print(rgb)

rgb['01101111', '01000010', '00111100']
binC 01101111
binC last two: 11
binC 01000010
binC last two: 10
binC 00111100
binC last two: 00
111000
